## Xception model training

### import libs

In [ ]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

In [ ]:
nb_classes = 20
nb_gpus = 4
image_size = (299, 299)
input_shape= (299,299,3)

train_path = "/home/cnn/Documents/tct/conference/xception/batch6_cells_half299/train/"
valid_path = "/home/cnn/Documents/tct/conference/xception/batch6_cells_half299/valid/"

### pretrain model

In [ ]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
parallel_model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 64
epochs = 1

In [ ]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator))

model.save_weights("Xception_first_train.h5")

### train model

In [ ]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('Xception_first_train.h5')

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
parallel_model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 64
epochs = 100

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

img_gen_t = ImageDataGenerator(rotation_range=90,                            
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               brightness_range=[0.9, 1.1],
#                               zca_whitening= True,
                               horizontal_flip=True,
                               vertical_flip=True)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

checkpoint = ModelCheckpoint("weights_{epoch:03d}_{val_loss:.4f}.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=False, save_weights_only=True, mode='min', period=1)

tensorboard = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)

callbacks = [checkpoint, tensorboard]

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator), 
                             callbacks=callbacks)

#### plot training trend

In [ ]:
# 1. first need to retrieve acc/loss/val_acc/val_loss info from tensorboard
# in terminal: tensorboard --logdir ./logs
# open browser with given link
# save data to local csv and merge four separate files

# 2. read from saved data csv
import pandas as pd
import matplotlib.pyplot as plt

csv_file = "./info_20181118.csv"

df = pd.read_csv(csv_file)

step = list(df.step)
acc = list(df.acc)
loss = list(df.loss)
val_acc = list(df.val_acc)
val_loss = list(df.val_loss)

# 3. plot acc and loss
from matplotlib.patches import Rectangle

fig = plt.figure(1, figsize=(12,6), dpi=90)
ax = fig.add_subplot(121)
ax.plot(step, acc,label="train")
ax.plot(step, val_acc,label="valid")
ax.legend()
plt.title("training accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
ax = fig.add_subplot(122)
ax.plot(step, loss,label="train")
ax.plot(step, val_loss,label="vaild")
ax.legend()
plt.title("training loss")
plt.xlabel("epoch")
plt.ylabel("loss")